In [2]:
audio_path = "/content/city_sounds.mp3"

In [3]:
# Installing the Libraries

!pip uninstall --quiet whisper -y
!pip uninstall --quiet openai-whisper -y
!pip install --quiet panns-inference openai-whisper torch torchaudio groq librosa numpy requests pydub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 19.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 13.3 MB/s eta 0:00:00


In [20]:
# API Keys
import os
from dotenv import load_dotenv

load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
ELEVENLABS_API_KEY = os.getenv("ELEVENLABS_API_KEY")
RUNWAYML_API_SECRET = os.getenv("RUNWAYML_API_SECRET")

In [5]:
import torch
import torchaudio
import whisper
from panns_inference import AudioTagging, labels
import librosa
import numpy as np
import requests
import json
from groq import Groq
from pydub import AudioSegment

/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


In [6]:
# Initialize models
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using: {device}\n")
panns_model = AudioTagging(checkpoint_path=None, device=device)
whisper_model = whisper.load_model("base", device=device)

Using: cuda

Checkpoint path: /root/panns_data/Cnn14_mAP=0.431.pth
GPU number: 1


100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 183MiB/s]


In [7]:
# Load and prepare audio
audio_path = "/content/city_sounds.mp3"
waveform, sr = torchaudio.load(audio_path)
if sr != 32000:
    waveform = torchaudio.transforms.Resample(orig_freq=sr, new_freq=32000)(waveform)
if waveform.shape[0] > 1:
    waveform = waveform.mean(dim=0, keepdim=True)
waveform = waveform.to(device)

/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be r

In [8]:
# Get sound tags
clipwise_output, embedding = panns_model.inference(waveform)
clipwise_output = clipwise_output.squeeze()
top_indices = clipwise_output.argsort()[-5:][::-1]
sound_types = [labels[int(i)] for i in top_indices]
print("🔊 Detected sounds:")
for i in top_indices:
    print(f"  • {labels[int(i)]}: {clipwise_output[int(i)]:.2%}")

🔊 Detected sounds:
  • Speech: 70.71%
  • Vehicle: 61.40%
  • Car: 25.56%
  • Outside, urban or manmade: 13.38%
  • Traffic noise, roadway noise: 13.34%


In [9]:
# Get speech transcript via whisper open ai
print("\n🎤 Transcribing speech...")
result = whisper_model.transcribe(audio_path)
transcript = result["text"].strip()
print(f"  💬 \"{transcript}\"" if transcript else "  (no speech)")


🎤 Transcribing speech...
  💬 "1.5% 1.5% 1.5% 1.5% 1.5%"


In [16]:
# Generate narration with Groq
client = Groq(api_key=GROQ_API_KEY)  # Replace with your Groq API key

sound_str = ", ".join(sound_types)

prompt = (
f"Write a lively, conversational 2-3 sentence narration of a bustling city scene, capturing the urban vibe based on sounds: {sound_str}. Seamlessly integrate a passerby or vendor casually saying '{transcript}' as part of the scene, using square brackets (e.g., [thoughtfully], [sarcastically], [enthusiastically]) to denote emotional voice tones for expressive dialogue delivery. Ensure the narration is natural, immersive, and reflects the full essence of the audio through vivid emotional expression, without including sound effect descriptions in the text."
)

response = client.chat.completions.create(
    messages=[{"role": "user", "content": prompt}],
    model="llama-3.3-70b-versatile",
    temperature=0.4,  # Lower for natural, focused tone
    max_tokens=100  # Ensure no cutoff
)
narration = response.choices[0].message.content
with open("narration.txt", "w") as f:
    f.write(narration)
print(f"\n✨ Narration: {narration}")


✨ Narration: As people rushed to and fro, a vendor [excitedly] called out to passersby, "1.5% 1.5% 1.5% 1.5% 1.5%," trying to entice them with a great deal, while a pedestrian [thoughtfully] muttered to herself about the latest interest rates. The city was alive with the hum of activity, as strangers became temporary companions in the shared experience of navigating the crowded streets. Amidst


In [27]:
import requests
from pydub import AudioSegment
import os

url = "https://api.elevenlabs.io/v1/text-to-speech/cgSgspJ2msm6clMCkdW9"  # Jessica voice
headers = {
    "xi-api-key": ELEVENLABS_API_KEY,
    "Content-Type": "application/json",
    "Accept": "audio/mpeg"
}
payload = {
    "text": narration,  # model's narration with emotional tags
    "model_id": "eleven_v3",  # v3 for emotional tags
    "voice_settings": {
        "stability": 0.0,  # Creative mode for emotional expressiveness
        "similarity_boost": 0.85,  # Enhance voice clarity
        "style": 0.6,  # Balanced for emotional nuance
        "use_speaker_boost": True  # Boost clarity for Jessica’s voice
    }
}

# Send request to ElevenLabs API
try:
    response = requests.post(url, headers=headers, json=payload, timeout=30)
    if response.status_code == 200:
        # Save temporary audio
        with open("temp_output.mp3", "wb") as f:
            f.write(response.content)

        # Amplify and normalize audio with pydub
        audio = AudioSegment.from_mp3("temp_output.mp3")
        audio = audio + 8  # 8dB boost for balance
        audio = audio.normalize()  # Prevent clipping
        audio.export("output.mp3", format="mp3", bitrate="192k")  # High-quality output
        os.remove("temp_output.mp3")  # Clean up
        print("✅ Audio saved as output.mp3")
    else:
        print(f"Error: {response.status_code} - {response.text}")
except Exception as e:
    print(f"Request failed: {str(e)}")

✅ Audio saved as output.mp3
